# Markov Chain Attribution Modeling

**Resources**

- https://medium.com/@akanksha.etc302/python-implementation-of-markov-chain-attribution-model-0924687e4037
- https://github.com/jerednel/markov-chain-attribution

In [68]:
# load modules and packages
import numpy as np
import pandas as pd
import copy

In [71]:
# import data & make a copy *avoid overwritting data
#df = pd.read_csv("C:/Users/tatel/OneDrive/Documents/Rstudio/datasets/attribution_data.csv")
# URL to our data source
url = "https://raw.githubusercontent.com/jrdnlwry/markov-chain-attribution/main/attribution_data.csv"
df = pd.read_csv(url)

display(df.head(n=5))

#df_copy = copy.deepcopy(df)

# convert time to timestamp from object data type
df["time"] = pd.to_datetime(df["time"])

,cookie,time,interaction,conversion,conversion_value,channel
0,00000FkCnDfDDf0iC97iC703B,2018-07-03T13:02:11Z,impression,0,0.0,Instagram
1,00000FkCnDfDDf0iC97iC703B,2018-07-17T19:15:07Z,impression,0,0.0,Online Display
2,00000FkCnDfDDf0iC97iC703B,2018-07-24T15:51:46Z,impression,0,0.0,Online Display
3,00000FkCnDfDDf0iC97iC703B,2018-07-29T07:44:51Z,impression,0,0.0,Online Display
4,0000nACkD9nFkBBDECD3ki00E,2018-07-03T09:44:57Z,impression,0,0.0,Paid Search


In [72]:
def calculate_duration_impression(dataFrame):

    df = copy.deepcopy(dataFrame)
    
    # Calculate the time duration from impression -> purchase
    
    # sort by cookie and time
    df = df.sort_values(by=["cookie", "time"])
    # for each cookie, get the first impression and first conversion time
    first_impression = df[df["interaction"] == "impression"].groupby("cookie")["time"].min().reset_index()
    # first conversion time
    first_conversion = df[df["conversion"] == 1].groupby("cookie")["time"].min().reset_index()
    
    # merge to get impression -> conversion duration
    merged_df = pd.merge(first_impression, first_conversion, on="cookie", suffixes=("_impression", "_conversion"))
    
    # calculate the duration
    merged_df["time_to_conversion"] = merged_df["time_conversion"] - merged_df["time_impression"]
    
    # filter for row with a positive time difference
    merged_df = merged_df[merged_df["time_to_conversion"] > pd.Timedelta(0)]

    return merged_df

duration_df = calculate_duration_impression(df_copy)
display(duration_df.head())

,cookie,time_impression,time_conversion,time_to_conversion
0,000hCBnCB7oi7ADAEnEBCnBEE,2018-07-07 14:34:54+00:00,2018-07-25 11:15:16+00:00,17 days 20:40:22
1,000kiDB3D0fCfDAohCDB3ohko,2018-07-15 19:17:09+00:00,2018-07-26 16:16:21+00:00,10 days 20:59:12
2,0033CAi9nhFf0DAA7DDCC3FAn,2018-07-08 09:02:08+00:00,2018-07-25 20:46:15+00:00,17 days 11:44:07
3,009B3hhBfAfhDCEEFEnB9ABBB,2018-07-22 10:29:36+00:00,2018-07-22 10:29:48+00:00,0 days 00:00:12
4,00ADnBD9B979fE0fnoB9CA0E9,2018-07-04 02:39:10+00:00,2018-07-07 22:07:40+00:00,3 days 19:28:30


The average time from impression to purchase is 8 days.

Our customer journey is defined from impression to purchase over the course of 8 days.

In [74]:
# display the overall average purchase time
duration_df["time_to_conversion"].mean()

Timedelta('8 days 05:55:42.256704980')

## Data Prep

- Package currently accepts a single-column Pandas dataframe
- Each path should begin with 'start' and end with either "conv" or "null"
    "null" indicates a conversion did not happen
- Each path should be delimited by ">"

The argument to pass is `paths`, where paths is the Pandas dataframe containing the conversion paths.

In [79]:
def data_prep(dataframe):
    
    df_copy = copy.deepcopy(dataframe)
    # Sort and create visit order
    df_sort = df_copy.sort_values(["cookie", "time"], ascending=[False, True])
    df_sort["visit_order"] = df_sort.groupby('cookie').cumcount() + 1
    
    # Get final interaction per user and merge it
    df_last_interaction = df_sort.drop_duplicates('cookie', keep='last')[['cookie', 'conversion']]
    df_paths = df_sort.groupby('cookie')['channel'].apply(list).reset_index()
    df_paths = pd.merge(df_paths, df_last_interaction, on='cookie')
    
    # Build path string
    df_paths['path'] = df_paths.apply(
        lambda row: ['start'] + row['channel'] + ['conv' if row['conversion'] == 1 else 'null'],
        axis=1
    )
    
    #display(df_paths)
    # join the list elements together into a single string
    df_paths["paths"] = df_paths["path"].apply(lambda x: ' > '.join(x))


    return df_paths

paths_df = data_prep(df)
display(paths_df)

,cookie,channel,conversion,path,paths
0,00000FkCnDfDDf0iC97iC703B,"[Instagram, Online Display, Online Display, On...",0,"[start, Instagram, Online Display, Online Disp...",start > Instagram > Online Display > Online Di...
1,0000nACkD9nFkBBDECD3ki00E,"[Paid Search, Paid Search, Paid Search, Paid S...",0,"[start, Paid Search, Paid Search, Paid Search,...",start > Paid Search > Paid Search > Paid Searc...
2,0003EfE37E93D0BC03iBhBBhF,"[Paid Search, Paid Search, Paid Search, Paid S...",0,"[start, Paid Search, Paid Search, Paid Search,...",start > Paid Search > Paid Search > Paid Searc...
3,00073CFE3FoFCn70fBhB3kfon,[Instagram],0,"[start, Instagram, null]",start > Instagram > null
4,00079hhBkDF3k3kDkiFi9EFAD,[Paid Search],0,"[start, Paid Search, null]",start > Paid Search > null
...,...,...,...,...,...
240103,ooooE0hkAFBkED90ChDDiBFAf,[Online Display],0,"[start, Online Display, null]",start > Online Display > null
240104,ooooEBE0o0D97ACAAAnDoi3F0,[Online Display],0,"[start, Online Display, null]",start > Online Display > null
240105,ooooEiB0CCoEf9fiiC90Dfhfk,"[Online Display, Online Display, Online Display]",0,"[start, Online Display, Online Display, Online...",start > Online Display > Online Display > Onli...
240106,ooooiBh70D3k3BfAhDFfii9h7,"[Paid Search, Online Video]",0,"[start, Paid Search, Online Video, null]",start > Paid Search > Online Video > null


**Sample our data**

We want to randomly sample the first 500 observations to avoid crashing the computer when trying to build the Markov Chain model.

Moving forward, to reduce computational time this can be converted into a sparse matrix.

In [80]:
# convert the paths column into a dataframe
paths = df_paths["paths"].to_frame()
# I only want to run this on the first 500 randomly sampled rows otherwise it will crash the computer
sample_paths = paths.sample(n=500, random_state=42)   # random state allows reproducibity

sample_paths = sample_paths.reset_index(drop=True)

len(sample_paths)

500

In [77]:
# # Example: check if 'conv' appears in the 'path' column
# matches = sample_paths[sample_paths['paths'].str.contains('conv', case=False, na=False)]

# # Print how many rows contain 'conv'
# print(f"Rows containing 'conv': {len(matches)}")

# # Optionally print the first few matching rows
# print(matches.head())

Run and display the Markov Model

In [81]:
model = run_model(paths=sample_paths)
# displays the overall model output
display(model)

{'markov_conversions': {'Facebook': 13.098106783746228,
  'Instagram': 6.259572457543932,
  'Online Display': 4.618791849216405,
  'Online Video': 3.36711834330141,
  'Paid Search': 7.656410566192019},
 'last_touch_conversions': {'Facebook': 15,
  'Online Video': 3,
  'Online Display': 5,
  'Paid Search': 8,
  'Instagram': 4},
 'removal_effects': {'Facebook': 0.5140382759135721,
  'Instagram': 0.24565839072443962,
  'Online Display': 0.18126557052665881,
  'Online Video': 0.13214334991797805,
  'Paid Search': 0.30047763024925833},
 'base_cvr': 0.06999999999999999,
 'transition_matrix':                 Facebook  Instagram  Online Display  Online Video  \
 Facebook        0.433566   0.125874        0.006993      0.016317   
 Instagram       0.361290   0.154839        0.045161      0.025806   
 Online Display  0.051852   0.029630        0.288889      0.022222   
 Online Video    0.061728   0.012346        0.037037      0.512346   
 Paid Search     0.054795   0.020548        0.027397      

In [82]:
print(model["markov_conversions"])

{'Facebook': 13.098106783746228, 'Instagram': 6.259572457543932, 'Online Display': 4.618791849216405, 'Online Video': 3.36711834330141, 'Paid Search': 7.656410566192019}


In [83]:
display(model["removal_effects"])

{'Facebook': 0.5140382759135721,
 'Instagram': 0.24565839072443962,
 'Online Display': 0.18126557052665881,
 'Online Video': 0.13214334991797805,
 'Paid Search': 0.30047763024925833}

### Removal Effects

The Markov Chain provides a way to model the conversion funnel.

In order to go from Markov Model to Attribution logic, we compute removal effects of each channel.

This is the differencee in conversions if that channel happens to be removed.

- Facebook: removing Facebook would reduce overall conversions by ~51% indicating it is likely a critical channel in the above customer journey
- Instagram:  conversions would drop ~25% without Instagram indicating it is moderately influential
- Online Display: around 18% drop if this channel is removed. It is impactful, possibly indicating it is an upper funnel channel.
- Online Video: contributes less than others, but still helps about ~13% of conversions.
- Paid Search: a 30% drop implies Paid Search plays a strong closer or decision role in the funnel.

Checking the logic this makes sense as on average it takes approximately 8 days before a conversion takes place indicating many touch points between that time period.

## Markov Chain Package

- Package currently accepts a single-column Pandas dataframe
- Each path should begin with 'start' and end with either "conv" or "null"
    "null" indicates a conversion did not happen
- Each path should be delimited by ">"

The argument to pass is `paths`, where paths is the Pandas dataframe containing the conversion paths.

In [84]:
import pandas as pd
import numpy as np
import re

def run_model(paths):
    regex = re.compile('[^a-zA-Z> ]')
    paths.rename(columns={paths.columns[0]: "Paths"}, inplace=True)
    paths['Paths'] = paths['Paths'].apply(lambda x: regex.sub('', x))
    return first_order(paths)

def calculate_removals(df, base_cvr):
    removal_effect_list = dict()
    channels_to_remove = df.drop(['conv', 'null', 'start'], axis=1).columns
    for channel in channels_to_remove:
        removal_df = df.drop(index=channel, columns=channel).copy()
        for col in removal_df.columns:
            row_sum = removal_df.loc[col].sum()
            null_percent = 1.0 - row_sum
            if null_percent > 0:
                removal_df.loc[col, 'null'] = null_percent
        removal_df.loc['null', 'null'] = 1.0
        R = removal_df[['null', 'conv']].drop(index=['null', 'conv'], errors='ignore')
        Q = removal_df.drop(columns=['null', 'conv'], errors='ignore').drop(index=['null', 'conv'], errors='ignore')

        try:
            N = np.linalg.inv(np.identity(len(Q)) - Q.to_numpy())
            M = np.dot(N, R.to_numpy())
            removal_cvr = pd.DataFrame(M, index=R.index)[[1]].loc['start'].values[0]
            removal_effect = 1 - removal_cvr / base_cvr if base_cvr != 0 else 0
            removal_effect_list[channel] = removal_effect
        except Exception:
            removal_effect_list[channel] = 0
    return removal_effect_list

def first_order(paths_df):
    paths = paths_df["Paths"].tolist()
    parsed_paths = [p.split(' > ') for p in paths]

    unique_touch_list = set(x for path in parsed_paths for x in path)
    total_conversions = sum(1 for path in parsed_paths if 'conv' in path)
    total_paths = len(paths)

    conv_dict = {tp: 0 for tp in unique_touch_list}
    for path in parsed_paths:
        if 'conv' in path and len(path) >= 2:
            conv_dict[path[-2]] += 1

    base_cvr = total_conversions / total_paths if total_paths > 0 else 0
    transitionStates = {f"{x}>{y}": 0 for x in unique_touch_list for y in unique_touch_list}

    for path in parsed_paths:
        for i in range(len(path) - 1):
            transitionStates[f"{path[i]}>{path[i+1]}"] += 1

    actual_paths = []
    for state in unique_touch_list - {'null', 'conv'}:
        state_transitions = {k: v for k, v in transitionStates.items() if k.startswith(f"{state}>") and v > 0}
        total = sum(state_transitions.values())
        for k, v in state_transitions.items():
            actual_paths.append({k: v / total if total > 0 else 0})

    flattened_matrix = [item for sublist in actual_paths for item in sublist.items()]
    trans_df = pd.DataFrame(flattened_matrix, columns=['paths', 'prob'])
    trans_df = trans_df.join(trans_df['paths'].str.split('>', expand=True).rename(columns={0: 'channel0', 1: 'channel1'}))

    states = sorted(unique_touch_list)
    test_df = pd.DataFrame(0.0, index=states, columns=states)
    for _, row in trans_df.iterrows():
        test_df.loc[row['channel0'], row['channel1']] = row['prob']
    test_df.loc['conv', 'conv'] = 1.0
    test_df.loc['null', 'null'] = 1.0

    R = test_df[['null', 'conv']].drop(index=['null', 'conv'], errors='ignore')
    Q = test_df.drop(columns=['null', 'conv'], errors='ignore').drop(index=['null', 'conv'], errors='ignore')

    try:
        N = np.linalg.inv(np.identity(len(Q)) - Q.to_numpy())
        M = np.dot(N, R.to_numpy())
        base_cvr = pd.DataFrame(M, index=R.index)[[1]].loc['start'].values[0]
    except Exception:
        M = np.zeros((len(Q), 2))
        base_cvr = 0

    removal_effects = calculate_removals(test_df.copy(), base_cvr)
    denominator = sum(removal_effects.values()) or 1e-9
    allocation_amount = [(effect / denominator) * total_conversions for effect in removal_effects.values()]

    markov_conversions = dict(zip(removal_effects.keys(), allocation_amount))

    for key in ['conv', 'null', 'start']:
        conv_dict.pop(key, None)

    return {
        'markov_conversions': markov_conversions,
        'last_touch_conversions': conv_dict,
        'removal_effects': removal_effects,
        'base_cvr': base_cvr,
        'transition_matrix': test_df,
        'absorption_matrix': M
    }
